In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_path = "../data/data_investeringer.xlsx"
df = pd.read_excel(data_path)
df = df[df['ISIN kode'].notna()]


In [ ]:
# Replace '-' with NaN (remove entries with no value)
df['Markedsværdi (DKK)'] = df['Markedsværdi (DKK)'].replace('-', np.nan)

# Remove any non-numeric characters except for digits and decimal points
df['Markedsværdi (DKK)'] = df['Markedsværdi (DKK)'].replace(r'[^\d.]', '', regex=True)

# Convert the column to float (after cleaning)
df['Markedsværdi (DKK)'] = pd.to_numeric(df['Markedsværdi (DKK)'], errors='coerce')

# Display the updated dataframe
df.head()


In [ ]:
data_path = "../data/pfa_eksklusionsliste.xlsx"
pfa = pd.read_excel(data_path)

In [ ]:
pfa = pfa[pfa['Land'].notna()]
pfa['Kilde til liste'] = 'PFA'
pfa

## Tilføj ISIN-numre til eksklusionsliste

In [ ]:
import pandas as pd
import re

# Sample data
df1 = df
df2 = pfa

# Normalize company names by converting to lowercase and removing special characters
def normalize_name(name):
    if pd.isna(name):
        return ""
    # Lowercase and remove special characters
    return re.sub(r'\W+', '', name.lower())

# Apply normalization to relevant columns
df1['Udsteder_normalized'] = df1['Udsteder'].apply(normalize_name)
df1['Værdipapirets navn_normalized'] = df1['Værdipapirets navn'].apply(normalize_name)
df2['Selskab_normalized'] = df2['Selskab'].apply(normalize_name)


In [ ]:
# Function to find ISINs, Udsteder, and Værdipapirets navn based on partial match
def find_isin_and_names(selskab, df1):
    matches = df1[
        (df1['Udsteder_normalized'].str.contains(selskab)) |
        (df1['Værdipapirets navn_normalized'].str.contains(selskab))
    ]
    # Return unique ISINs, Udsteder, and Værdipapirets navn if matches are found
    if not matches.empty:
        isins = matches['ISIN kode'].unique().tolist()
        udsteder = matches['Udsteder'].unique().tolist()
        værdipapirets_navn = matches['Værdipapirets navn'].unique().tolist()
        return isins, udsteder, værdipapirets_navn
    else:
        return [], [], []

# Apply the function to each row in df2 and create new columns for ISIN, Udsteder, and Værdipapirets navn
df2[['ISIN', 'Matched Udsteder', 'Matched Værdipapirets navn']] = df2['Selskab_normalized'].apply(
    lambda x: pd.Series(find_isin_and_names(x, df1))
)


In [ ]:
# Save df2 as an Excel file
# file_path = "../data/pfa_eksklutionsliste_isin.xlsx"
# df2.to_excel(file_path, index=False)

# Display the resulting dataframe
# import ace_tools as tools; tools.display_dataframe_to_user(name="Updated Second DataFrame with ISIN, Udsteder, and Værdipapirets navn", dataframe=df2)


In [ ]:
df2.head()

## Tilføj kolonner til oprindelig liste

In [ ]:
import pandas as pd

# Assume df1 and df2 are already defined

# Initialize new columns in df1 with empty strings
df1['Problematisk ifølge:'] = ""
df1['Forklaring'] = ""

# Iterate over each row in df2
for index, row in df2.iterrows():
    # Get the list of ISINs from df2
    isin_list = row['ISIN']
    kilde = row['Kilde til liste']
    forklaring = row['Årsag til eksklusion']
    
    # Find matching ISINs in df1
    df1_matches = df1[df1['ISIN kode'].isin(isin_list)]
    
    # Update df1 with the matching values from df2
    df1.loc[df1_matches.index, 'Problematisk ifølge:'] = kilde
    df1.loc[df1_matches.index, 'Forklaring'] = forklaring

In [ ]:
# Filter df1 to show only rows where there is a match
matched_rows = df1[df1['Problematisk ifølge:'] != ""]

matched_rows

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL of the exclusion list
url = "https://akademikerpension.dk/ansvarlighed/frasalg-og-eksklusion/"

# Send a request to fetch the webpage content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Define exclusion categories and their reasons
exclusion_categories = {
    'Kul': 'Kul',
    'Menneskerettigheder': 'Menneskerettigheder',
    'Olie og gas': 'Olie og gas',
    'Tobak': 'Tobak',
    'Tjæresand': 'Tjæresand',
    'Våben': 'Våben'
}

# Initialize empty lists to store the data
companies = []
reasons = []

# Loop through each exclusion category
for category, reason in exclusion_categories.items():
    # Find the section with the category title
    category_section = soup.find('h2', class_='card__title', text=lambda x: x and category in x)
    
    if category_section:
        # Find the list of companies in the next "ul" after the category
        company_list = category_section.find_next('ul')
        
        if company_list:
            for company in company_list.find_all('p', class_='navigation-item__link-title'):
                companies.append(company.text.strip())
                reasons.append(reason)

# Create a DataFrame with the scraped data
df = pd.DataFrame({
    'Company Name': companies,
    'Reason': reasons
})

# Save the DataFrame to an Excel file
file_path = 'akademikerpension_exclusions.xlsx'
df.to_excel(file_path, index=False)

file_path
